In [1]:
import os

import numpy as np
import random
import tensorflow as tf

# Set seeds for reproducibility
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'


# Function to append logs
def append_log(content):
    """Appends a string to the log file inside the model directory."""
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
    with open(log_file_path, "a", encoding="utf-8") as file:
        file.write(content + "\n")


/home/aymenbj/AERL39_2/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
import json
from sklearn.utils import shuffle
import os
import sys
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score,recall_score, precision_score
import pandas as pd
from tensorflow.keras.models import model_from_json

# Import definitions from your training file.
# It is assumed that this file defines the following:
#   - ismModifDoss, ismModif, ismModifDataset (used for file naming)
#   - RLenv: the environment class for loading/formatting data
#   - huber_loss: your custom loss function
# (Other classes such as DefenderAgent, etc., are defined but for AERL fine tuning
# we work directly with the defender’s Keras model.)

from gan_model import * 

import tensorflow.keras.backend as K  # ✅ Import K globally
from tensorflow.keras.models import load_model

/home/aymenbj/AERL39_2/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
!mkdir splitted/

In [2]:
train_path = "splitted/train.csv"
test_path = "splitted/test.csv"

In [23]:
names = ["frame.interface_id" ,"frame.dlt" ,"frame.offset_shift","frame.time_epoch" ,
        "frame.time_delta" ,"frame.time_delta_displayed" ,"frame.time_relative" ,
         "frame.len" ,"frame.cap_len" ,"frame.marked" ,"frame.ignored" ,"radiotap.version" ,
    "radiotap.pad" ,"radiotap.length" ,"radiotap.present.tsft" ,"radiotap.present.flags" ,"radiotap.present.rate" ,
    "radiotap.present.channel" ,"radiotap.present.fhss" ,"radiotap.present.dbm_antsignal" ,"radiotap.present.dbm_antnoise" ,
    "radiotap.present.lock_quality" ,"radiotap.present.tx_attenuation" ,"radiotap.present.db_tx_attenuation" ,
    "radiotap.present.dbm_tx_power" ,"radiotap.present.antenna" ,"radiotap.present.db_antsignal" ,"radiotap.present.db_antnoise" ,
    "radiotap.present.rxflags" ,"radiotap.present.xchannel" ,"radiotap.present.mcs" ,"radiotap.present.ampdu" ,"radiotap.present.vht" ,
    "radiotap.present.reserved" ,"radiotap.present.rtap_ns" ,"radiotap.present.vendor_ns" ,"radiotap.present.ext" ,
    "radiotap.mactime" ,"radiotap.flags.cfp" ,"radiotap.flags.preamble" ,"radiotap.flags.wep" ,"radiotap.flags.frag" ,
    "radiotap.flags.fcs" ,"radiotap.flags.datapad" ,"radiotap.flags.badfcs" ,"radiotap.flags.shortgi" ,"radiotap.datarate" ,
    "radiotap.channel.freq" ,"radiotap.channel.type.turbo" ,"radiotap.channel.type.cck" ,"radiotap.channel.type.ofdm" ,
    "radiotap.channel.type.2ghz" ,"radiotap.channel.type.5ghz" ,"radiotap.channel.type.passive" ,"radiotap.channel.type.dynamic" ,
    "raÇdiotap.channel.type.gfsk" ,"radiotap.channel.type.gsm" ,"radiotap.channel.type.sturbo" ,"radiotap.channel.type.half" ,
    "radiotap.channel.type.quarter" ,"radiotap.dbm_antsignal" ,"radiotap.antenna" ,"radiotap.rxflags.badplcp" ,"wlan.fc.type_subtype" ,
    "wlan.fc.version" ,"wlan.fc.type" ,"wlan.fc.subtype" ,"wlan.fc.ds" ,"wlan.fc.frag" ,"wlan.fc.retry" ,"wlan.fc.pwrmgt" ,
    "wlan.fc.moredata" ,"wlan.fc.protected" ,"wlan.fc.order" ,"wlan.duration" ,"wlan.ra" ,"wlan.da" ,"wlan.ta" ,"wlan.sa" ,
    "wlan.bssid" ,"wlan.frag" ,"wlan.seq" ,"wlan.bar.type" ,"wlan.ba.control.ackpolicy" ,"wlan.ba.control.multitid" ,
    "wlan.ba.control.cbitmap" ,"wlan.bar.compressed.tidinfo" ,"wlan.ba.bm" ,"wlan.fcs_good" ,"wlan_mgt.fixed.capabilities.ess" ,
    "wlan_mgt.fixed.capabilities.ibss" ,"wlan_mgt.fixed.capabilities.cfpoll.ap" ,"wlan_mgt.fixed.capabilities.privacy" ,
    "wlan_mgt.fixed.capabilities.preamble" ,"wlan_mgt.fixed.capabilities.pbcc" ,"wlan_mgt.fixed.capabilities.agility" ,
    "wlan_mgt.fixed.capabilities.spec_man" ,"wlan_mgt.fixed.capabilities.short_slot_time" ,"wlan_mgt.fixed.capabilities.apsd" ,
    "wlan_mgt.fixed.capabilities.radio_measurement" ,"wlan_mgt.fixed.capabilities.dsss_ofdm" ,"wlan_mgt.fixed.capabilities.del_blk_ack" ,
    "wlan_mgt.fixed.capabilities.imm_blk_ack" ,"wlan_mgt.fixed.listen_ival" ,"wlan_mgt.fixed.current_ap" ,"wlan_mgt.fixed.status_code" ,
    "wlan_mgt.fixed.timestamp" ,"wlan_mgt.fixed.beacon" ,"wlan_mgt.fixed.aid" ,"wlan_mgt.fixed.reason_code" ,"wlan_mgt.fixed.auth.alg" ,
    "wlan_mgt.fixed.auth_seq" ,"wlan_mgt.fixed.category_code" ,"wlan_mgt.fixed.htact" ,"wlan_mgt.fixed.chanwidth" ,
    "wlan_mgt.fixed.fragment" ,"wlan_mgt.fixed.sequence" ,"wlan_mgt.tagged.all" ,"wlan_mgt.ssid" ,"wlan_mgt.ds.current_channel" ,
    "wlan_mgt.tim.dtim_count" ,"wlan_mgt.tim.dtim_period" ,"wlan_mgt.tim.bmapctl.multicast" ,"wlan_mgt.tim.bmapctl.offset" ,
    "wlan_mgt.country_info.environment" ,"wlan_mgt.rsn.version" ,"wlan_mgt.rsn.gcs.type" ,"wlan_mgt.rsn.pcs.count" ,
    "wlan_mgt.rsn.akms.count" ,"wlan_mgt.rsn.akms.type" ,"wlan_mgt.rsn.capabilities.preauth" ,"wlan_mgt.rsn.capabilities.no_pairwise" ,
    "wlan_mgt.rsn.capabilities.ptksa_replay_counter" ,"wlan_mgt.rsn.capabilities.gtksa_replay_counter" ,"wlan_mgt.rsn.capabilities.mfpr" ,
    "wlan_mgt.rsn.capabilities.mfpc" ,"wlan_mgt.rsn.capabilities.peerkey" ,"wlan_mgt.tcprep.trsmt_pow" ,"wlan_mgt.tcprep.link_mrg" ,
    "wlan.wep.iv" ,"wlan.wep.key" ,"wlan.wep.icv" ,"wlan.tkip.extiv" ,"wlan.ccmp.extiv" ,"wlan.qos.tid" ,"wlan.qos.priority" ,
    "wlan.qos.eosp" ,"wlan.qos.ack" ,"wlan.qos.amsdupresent" ,"wlan.qos.buf_state_indicated" ,"wlan.qos.bit4" ,
    "wlan.qos.txop_dur_req" ,"wlan.qos.buf_state_indicated2" ,"data.len" ,"class"]

In [19]:
colnames_good_df = pd.read_csv("colnames_good.csv",names=["colnames"])
colnames_good = colnames_good_df['colnames'].tolist()

In [29]:
train = pd.read_csv("../datasets/awidclsr/AWID-CLS-R-Trn/AWID-CLS-R-Trn/1", names=names, usecols=colnames_good)
test = pd.read_csv("../datasets/awidclsr/AWID-CLS-R-Tst/AWID-CLS-R-Tst/1", names=names, usecols=colnames_good)

In [30]:
print(train.shape)
print(test.shape)

(1795575, 49)
(575643, 49)


In [31]:
train_indx = train.shape[0]

df = train.append(test, ignore_index=True)

# Remove labels column
labels = df['class']
df = df.drop('class',axis=1)

print("Dataset without labels shape:{}".format(df.shape))


# Processing categorical and numerical columns
num_cols = list(df._get_numeric_data().columns)
cat_cols = list(set(df.columns)-set(num_cols))


# Normalization of the df
#log_cols = self.df.filter(like='logarithm').columns
#nat_cols =  list(set(self.df.columns)-set(log_cols))
#self.df.reset_index()
df[num_cols] = (df[num_cols]-df[num_cols].min())/(df[num_cols].max()-df[num_cols].min())


for name_col in cat_cols:
    df = pd.concat([df.drop(name_col, axis=1), pd.get_dummies(df[name_col],prefix=name_col)], axis=1)

print("Dataset with categoricals encoded:{}".format(df.shape))

# If na max and min = 0 so delete column
df = df.dropna(axis=1)

# Delete row
#self.df = self.df.dropna(axis=0)
print("Dataset dropping nan:{}".format(df.shape))

# Add labels again
df['labels']=labels


print("Dataset with labels again:{}".format(df.shape))


train = df.iloc[:train_indx]
test = df.iloc[train_indx:]

Dataset without labels shape:(2371218, 48)
Dataset with categoricals encoded:(2371218, 70)
Dataset dropping nan:(2371218, 46)
Dataset with labels again:(2371218, 47)


In [32]:
print(train.shape)
print(test.shape)

(1795575, 47)
(575643, 47)


In [33]:
train.to_csv(train_path,index=False)
test.to_csv(test_path,index=False)

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795575 entries, 0 to 1795574
Data columns (total 47 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   frame.time_epoch                      float64
 1   frame.time_delta                      float64
 2   frame.time_delta_displayed            float64
 3   frame.time_relative                   float64
 4   frame.len                             float64
 5   frame.cap_len                         float64
 6   radiotap.length                       float64
 7   radiotap.present.tsft                 float64
 8   radiotap.present.flags                float64
 9   radiotap.present.channel              float64
 10  radiotap.present.dbm_antsignal        float64
 11  radiotap.present.antenna              float64
 12  radiotap.present.rxflags              float64
 13  radiotap.datarate                     float64
 14  wlan.fc.type                          float64
 15  wlan.fc.subtype

In [35]:
train.describe()

,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.channel,...,wlan.fc.type_subtype_0x1b,wlan.fc.type_subtype_0x1c,wlan.fc.type_subtype_0x1d,wlan.fc.type_subtype_0x20,wlan.fc.type_subtype_0x24,wlan.fc.type_subtype_0x28,wlan.fc.type_subtype_0x2c,wlan.fc.ds_0x00,wlan.fc.ds_0x01,wlan.fc.ds_0x02
count,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,...,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06,1.795575e+06
mean,2.312808e-01,5.723385e-03,5.723385e-03,5.195880e-01,3.106633e-01,3.106633e-01,9.995656e-01,9.995656e-01,9.995656e-01,9.995656e-01,...,7.100789e-04,1.999694e-02,2.321902e-01,1.302697e-01,1.033875e-02,3.683216e-01,5.524693e-04,4.905175e-01,1.369093e-01,3.725731e-01
std,1.072683e-01,1.821356e-02,1.821356e-02,2.409856e-01,4.345912e-01,4.345912e-01,2.083777e-02,2.083777e-02,2.083777e-02,2.083777e-02,...,2.663785e-02,1.399895e-01,4.222298e-01,3.366000e-01,1.011527e-01,4.823494e-01,2.349818e-02,4.999102e-01,3.437517e-01,4.834899e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.271072e-01,8.993242e-04,8.993242e-04,2.855550e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.352776e-01,1.567394e-03,1.567394e-03,5.285670e-01,4.705882e-02,4.705882e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.121584e-01,3.177612e-03,3.177612e-03,7.012848e-01,9.882353e-01,9.882353e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
max,4.451235e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575643 entries, 1795575 to 2371217
Data columns (total 47 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   frame.time_epoch                      575643 non-null  float64
 1   frame.time_delta                      575643 non-null  float64
 2   frame.time_delta_displayed            575643 non-null  float64
 3   frame.time_relative                   575643 non-null  float64
 4   frame.len                             575643 non-null  float64
 5   frame.cap_len                         575643 non-null  float64
 6   radiotap.length                       575643 non-null  float64
 7   radiotap.present.tsft                 575643 non-null  float64
 8   radiotap.present.flags                575643 non-null  float64
 9   radiotap.present.channel              575643 non-null  float64
 10  radiotap.present.dbm_antsignal        575643 non-null  float6

In [37]:
num_cols = list(train._get_numeric_data().columns)
cat_cols = list(set(train.columns)-set(num_cols))

In [38]:
len(num_cols)

46

In [39]:
train.shape

(1795575, 47)